In [3]:
import tensorflow as tf
import pandas as pd
import newlinejson as nlj
from glob import glob
from collections import defaultdict
import json
import os
import sys
import numpy as np 
import math
%matplotlib inline
import matplotlib.pyplot as plt
import concurrent.futures
import os.path
import pickle
plt.style.use('seaborn-whitegrid')


# for root, dirs, files in os.walk("C:\\Data\\quickdraw_simplified"):  
#     for filename in files:
#         print(filename.split('.')[0])

def save_obj(path,obj, name ):
    with open(path+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path ,name ):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
#test load object and print

key = 'bicycle'
obj = load_obj('C:\\Data\\Batch\\Bactch1\\Output\\',key)
print(len(obj))
for i in range(len(obj)):
    array = np.asarray(obj[i])
    print(array)    
        

In [ ]:

#with nlj.open('C:\\Data\\quickdraw_simplified\\airplane.ndjson') as src:

#files = glob('C:\\Data\\samples\\*.ndjson')

files = glob('C:\\Data\\Batch\\Bactch1\\*.ndjson')
maxWidth=0
maxHeight=0 
data = defaultdict(list)
for file in files:
    with nlj.open(file) as src:
        print(file)
        print(data.keys())
        for i, line in enumerate(src):
            if data.get(line['word']):
                 data[line['word']].append(line['drawing'])
            else:
                 data[line['word']] = [line['drawing']]
      

#these many objects
print(data.keys())

for key in data.keys():
    for i in range(len(data[key])):
        array = np.asarray(data[key][i])
        if array.ndim == 3 and i < 1000:
           # print(array.shape)
           # print(array)
            k=0
            plt.figure()
            print("scaled")
            for depth in range(array.shape[0]):
                xScaledValues = [round(x*0.1) for x in array[depth,0,0:]]
                yScaledValue = [round(y*0.1) for y in  array[depth,1,0:]]
                plt.plot(xScaledValues, yScaledValue, '-o', color='black');  
            plt.show()
            
            plt.figure()
            print("unscaled")
            for depth in range(array.shape[0]):
                plt.plot(array[depth,0,0:], array[depth,1,0:], '-o', color='black');  
            plt.show()
        elif array.ndim == 2 and i < 1000:
          #  print(array.shape)
          #   print(array)
            k=0
#             plt.figure()
#             print("scaled")
#             for row in range(array.shape[0]):
#                 xScaledValues = [x*0.1 for x in array[row,0]]
#                 yScaledValue = [y*0.1 for y in  array[row,1]]
#                 plt.plot(xScaledValues, yScaledValue, '-o', color='black');  
#             plt.show()
            
#             plt.figure()
#             for row in range(array.shape[0]): 
#                 plt.plot(array[row,0], array[row,1], '-o', color='black');  
#             plt.show()
        if array.ndim > 3:
                print("More dimentions")

In [ ]:
#Print all images
for key in data.keys():
    for i in range(len(data[key])):
        array = np.asarray(data[key][i])
        if array.ndim == 3 and i < 2000:
                # print(array.shape)
                # print(array)
                plt.figure()
                for row in range(array.shape[0]): 
                    plt.plot(array[row,0], array[row,1], '-o', color='black');  
                plt.show()
        elif array.ndim == 2 and i < 100:
                #print(array.shape[0])        
                #print(array)
                plt.figure()
                for row in range(array.shape[0]): 
                    plt.plot(array[row,0], array[row,1], '-o', color='black');  
                plt.show()


In [ ]:
#find maximum height and maximum widht in pixels

def FindMaxxMaxy(folderpath):
    data = defaultdict(list)
    #C:\\Data\\quickdraw_simplified\\*.ndjson
    files = glob(folderpath)
    maxWidth = 0
    maxHeight = 0
    tempMaxx = 0
    tempMaxy = 0
    for file in files:
        with nlj.open(file) as src:
            data = defaultdict(list)
            print(file)
            for i, line in enumerate(src):
                if data.get(line['word']):
                     data[line['word']].append(line['drawing'])
                else:
                     data[line['word']] = [line['drawing']]
   
    print("Calculating")  
    #Consume data object created obove
    for key in data.keys():
        for i in range(len(data[key])):
            array = np.asarray(data[key][i])
            if array.ndim == 3:
                for row in range(array.shape[0]): 
                    tempMaxx = max(array[row,0,0:])
                    tempMaxy = max(array[row,1,0:])
            elif array.ndim == 2: 
                for row in range(array.shape[0]): 
                     tempMaxx = max(array[row,0])
                     tempMaxy = max(array[row,1])
                        
                if maxWidth < tempMaxx:      
                     maxWidth = tempMaxx
                if maxHeight < tempMaxy:      
                     maxHeight = tempMaxy
           # print(maxHeight, "" , maxWidth)   
    
    print(maxHeight, "" , maxWidth)  
    return [maxWidth,maxHeight]
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=2)
a = executor.submit(FindMaxxMaxy,"C:\Data\Batch\Bactch1\\*.ndjson")
b = executor.submit(FindMaxxMaxy,"C:\Data\Batch\Bactch2\\*.ndjson")   
import json
x = json.dumps(data)
fh = open('C:\\Data\\Output\\data.json','w')
fh.write(x)     
          

In [ ]:

from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=2)
a = executor.submit(PrepareData,"C:\Data\Batch\Bactch1\\*.ndjson")

In [ ]:
#Prepare Matrix of size 255*255 and prepare data
from collections import defaultdict

def PrepareData(folderpath):
  
    #filespath = os.path.join(folderpath, '*.ndjson')
    imagedata = defaultdict(list)
    files = glob(folderpath)
    for file in files: 
        filename, file_extension = os.path.splitext(file)
        matrixList =[]
        with nlj.open(file) as src:
            print(file,filename)
            for i, line in enumerate(src):
                array = np.asarray(line['drawing'])
                matrix = np.array([[0 for i in range(27)] for i in range(27)])
                if array.ndim == 3: 
                        for depth in range(array.shape[0]): 
                            xScaledValues = [round(x*0.1,0) for x in array[depth,0,0:]]
                            yScaledValues = [round(y*0.1,0) for y in  array[depth,1,0:]]
                            for x,y in zip(xScaledValues, yScaledValue):
                                if x*y> 676:
                                    print(x,y)
                                matrix[int(x),int(y)]=255
                elif array.ndim == 2: 
                        for row in range(array.shape[0]): 
                            xScaledValues = [round(x*0.1) for x in array[row,0]]
                            yScaledValues = [round(y*0.1) for y in array[row,1]]
                            for x,y in zip(xScaledValues, yScaledValues):
                                if x*y> 676:
                                    print(x,y)
                                matrix[int(x),int(y)]=255

                imagedata[line['word']].append(matrix)
               
        #x = json.dumps(imagedata)
        #json.dump(imagedata, filename)
        fh = np.save(filename,imagedata)
        fh.write(x)

PrepareData("C:\\Data\\Batch\\Bactch1\\*.ndjson")